In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
isNotebook = '__file__' not in locals()

In [3]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *

In [4]:
from nlptools.topicmodeling import *
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import LdaMulticore
from gensim.test.utils import common_corpus, common_dictionary
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import gensim
from math import log2
from math import sqrt
from numpy import asarray

In [19]:
import pandas as pd
from IPython.display import display, HTML

In [5]:
logger = Logger(tmpDir('logs') + "/analysis.log") if isNotebook else Logger("analysis.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [6]:
twinewsRankings = getTwinewsRankings()

In [7]:
len(twinewsRankings)

11

In [66]:
def printResults(model=None, splitVersion=None, filt={}, noSubsampling=True, logger=None):
    data = []
    if noSubsampling and "maxUsers" not in filt:
        filt = mergeDicts(filt, {"maxUsers": None})
    if model is not None and "model" not in filt:
        filt = mergeDicts(filt, {"model": model})
    if splitVersion is not None and "splitVersion" not in filt:
        filt = mergeDicts(filt, {"splitVersion": splitVersion})
    for key in twinewsRankings.keys():
        toKeep = True
        meta = twinewsRankings.getMeta(key)
        for filtKey in filt:
            if filtKey not in meta:
                raise Exception(filtKey + "is not in " + b(meta, 5))
            if filt[filtKey] != meta[filtKey]:
                toKeep = False
                break
        if toKeep:
            data.append(meta)
    if len(data) == 0:
        log("No data found", logger)
    else:
        try:
            refKeys = data[0].keys()
            for e in data:
                assert e.keys() == refKeys
        except:
            raise Exception("Some data keys doesn't match:\n" + b(data, 5))
        if len(data) > 1:
            keysHavingSameValues = set(data[0].keys())
            baseValues = data[0]
            for current in data[1:]:
                for key in baseValues.keys():
                    if key in keysHavingSameValues and baseValues[key] != current[key]:
                        keysHavingSameValues.remove(key)
            sameValues = dict()
            for key in keysHavingSameValues:
                sameValues[key] = data[0][key]
            log("These values are common to all rows:\n", logger)
            for key, value in sameValues.items():
                log("\t- " + str(key) + ": " + str(value), logger)
            for i in range(len(data)):
                for key in keysHavingSameValues:
                    del data[i][key]
        df = pd.DataFrame(data)
        display(df)

In [68]:
printResults\
(
    model='nmf',
    splitVersion=2,
    noSubsampling=True,
    filt=\
    {
        # "implementation": "gensim-lda",
    },
    logger=logger,
)

These values are common to all rows:

	- lowercase: True
	- implementation: sklearn-nmf
	- nmfAlpha: 0.1
	- doLemmatization: True
	- maxDF: 300
	- nmfInit: nndsvd
	- model: nmf
	- minDF: 0.0005
	- splitVersion: 2
	- nmfL1Ratio: 0
	- distance: cosine
	- maxUsers: None
	- nbTopics: 30
	- useExtraNews: False


,maxDocuments,maxIter,historyRef,id
0,10000,200,0.3,nmf-dea50
1,1000,100,0.5,nmf-b61b9
2,1000,200,0.3,nmf-09f9c
